# **Convert Orthomosaic tiles from .TIF to .PNG for use in CoralNet**  
## Python Script to convert pre-processed orthomosaic (OM) constructed in Agisoft Metashape into format usable in CoralNet  
### This script tiles orthomosaics for processing imagery on CoralNet as these files are too large to upload to CoralNet.
### Methods applied in this script for OM tiling are:
#### 1) OM splitting according to a grid formatted in a shapefile created in ArcGIS Pro    
#### 2) OM splitting into identifically sized cells defined by number of pixels  
### The grid cells are approximately 1x1 m but not perfectly uniform in this specific case (project specific for the example orthomosaic)  

# Overview
In this notebook, you will pre-process benthic imagery for automatic classification and percent cover calculation of observed benthic categories using [CoralNet](https://coralnet.ucsd.edu/). CoralNet is an online, open-source machine learning classifier that enables the identification of benthic species and their relative abundance using an unsupervised species annotation algorithm and a predefined set of labels of species and bottom types as training data. In addition to the main trained classifier that automatically processes new images, there are a number of trained classifiers users can select for their images. These classifiers can be used to detect the presence of individual species in a reef habitat, and estimate percent cover of functional groups and benthic characteristics like bottom type in a automated fashion with minimum human effort. 

Benthic images may need to be pre-processed before ingestion into CoralNet. CoralNet restricts images to a maximum of 8000 x 8000 pixels, which prevents uploading large photomosaics. Orthomosaics are photomosaics generated when virtually reconstructing reefs in 3D with photogrammetric techniques, and can be used in conjunction with digital elevation models. Splitting a large orthomosaic or photomosaic into smaller images is necessary to use CoralNet, but a manual tiling process can be time consuming. This notebook has multiple methods to tile orthomosaics based on 1) [shapefile generated in GIS software](#1-the-om-is-split-according-to-a-grid-formatted-in-a-shapefile-created-in-arcgis-pro) and 2) [identically sized cells defined by number of pixels](#2-the-om-is-split-into-identifically-sized-cells-defined-by-number-of-pixels).

Benthic images may also include a physical photoquadrat to standardize data collection. This notebook also contains [Quadrant Recon](https://pypi.org/project/quadrantrecon/), an image recognition tool designed to crop quadrants from benthic imagery collected with photoquadrats as a preprocessing step for CoralNet. As configured by default, this tool will recognize yellow edges of quadrants collected following a standardized [protocol](https://repository.oceanbestpractices.org/handle/11329/2199.2) developed by the [MBON Pole to Pole Project](https://marinebon.github.io/p2p/index.html) on any file you select, and crop their inside area. The detection parameters can be configured to detect other colors, crop differently sized images, or even ignore different parts of the image to detect other quadrant-like shapes.





Load libraries

In [ ]:
import rasterio
import geopandas as gpd
from rasterio.windows import Window
import os



Load Orthomoasic in TIF format

In [ ]:
# Replace 'tif_path' with the path to your TIF file
tif_path = 'C:\\Users\\David.Kochan\\OneDrive - Florida Fish and Wildlife Conservation\\Desktop\\pinkgreen tiles\\04_pinkgreen_ortho_final.tif'
with rasterio.open(tif_path) as src:
    data = src.read()
    width = src.width
    height = src.height

1) If using a grid derived from a shapefile created in ArcGIS Pro complete the following 3 steps:  
1a. Load grid in the shapefile format  


In [ ]:
# Replace 'shapefile_path' with the path to your shapefile containing the grid 
# Make sure that all shapefile files are in the same folder (.shx, .sbn, etc)
shapefile_path = 'C:\\Users\\David.Kochan\\OneDrive - Florida Fish and Wildlife Conservation\\Desktop\\pinkgreen tiles\\grid_pinkgreen.shp'
grid = gpd.read_file(shapefile_path)


1b. Select output directory

In [ ]:
# Replace 'output_directory' with the path to the directory where you want to save the PNGs
# Will create folder if folder does not already exist
output_directory = 'C:\\Users\\David.Kochan\\OneDrive - Florida Fish and Wildlife Conservation\\Desktop\\pinkgreen pngs'
# Make sure the output directory exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

1c. Loop to tile orthomosaic according to the grid, naming tiles, and exporting tiles as PNG files 
  Each tile is being renamed according to the GRID column in the shapefile, which was created and named in ArcGIS Pro  
  For example, the first tile is named A1_pinkgreen.png.

  Note: Because of the orientation of the grid (starting in the top left), the row start and row stop indices are flipped  
  If you get an error "the window is not an integer or zero" flip the row or column starts and stops to ensure there is a positive value for the row and column indices

In [ ]:
# Loop through each grid feature and extract and save the corresponding sub-area as a PNG file.
for idx, feature in grid.iterrows():
    # Get the value from the specified column in the shapefile for naming the PNG file.
    name_column = 'GRID'  # Replace 'column_name' with the actual name of the column in your shapefile.
    png_name = str(feature[name_column]) + '_pinkgreen.png'
    # Get the coordinates of the bounding box for the grid cell.
    bounds = feature.geometry.bounds
    minx, miny, maxx, maxy = bounds

    # Calculate the row and column indices in the TIF file for the current grid cell.
    row_start, col_start = src.index(minx, miny)
    row_stop, col_stop = src.index(maxx, maxy)

    # Create a window to extract the sub-area
    window = Window.from_slices((row_stop, row_start+1), (col_start, col_stop+1))

    # Convert window attributes to integers.
    row_off = int(window.row_off)
    col_off = int(window.col_off)
    height = int(window.height)
    width = int(window.width)
    print(row_off)
    print(col_off)
    print(height)
    print(width)

    # Extract the sub-area from the TIF file.
    sub_data = data[:, row_off:row_off + height, col_off:col_off + width]


    # Get the output PNG filename (you can modify this based on your requirement).
    output_filename = os.path.join(output_directory, f'sub_area_{idx}.png')

    # Get the output PNG filename.
    output_filename = os.path.join(output_directory, png_name)

    # Save the sub-area as PNG
    with rasterio.open(output_filename, 'w', driver='PNG', width=sub_data.shape[2], height=sub_data.shape[1], count=sub_data.shape[0], dtype=sub_data.dtype) as dst:
        dst.write(sub_data)

2). If using a grid created by identical cells measured by pixels  
2a. Set pixel height and width  
For this example, using 1000 x 1000 pixels to represent 1x1 m cells (OM exported at 1mm pixel size).



In [ ]:
# Define cell dimensions
cell_width = 1000
cell_height = 1000

2b. Loop through the OM TIF and save each cell as a PNG

In [ ]:
# Loop through the TIF and extract and save each cell as PNG.
for y in range(0, height, cell_height):
    for x in range(0, width, cell_width):
        window = Window(x, y, min(cell_width, width - x), min(cell_height, height - y))
        sub_data = data[:, window.row_off:window.row_off + window.height, window.col_off:window.col_off + window.width]
        
        # Create a filename for the PNG based on cell position.
        cell_filename = f'cell_{y}_{x}.png'

        # Get the output PNG filename (you can modify this based on your requirement).
        output_filename = os.path.join(output_directory, cell_filename)
    
        
        # Save the cell as PNG.
        with rasterio.open(output_filename, 'w', driver='PNG', width=sub_data.shape[2], height=sub_data.shape[1], count=sub_data.shape[0], dtype=sub_data.dtype) as dst:
            dst.write(sub_data)